# Load libraries

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import Pool
from src.tools import *

import warnings
warnings.filterwarnings("ignore")

# Read data

In [2]:
df = pd.read_csv('../data/prepared_data.csv')
df_fe = pd.read_csv('../data/data_fe.csv')

# Train Catboost

## Test data without upsampling

### Find feature importance

In [3]:
CatBoost = CatBoostClassifier(verbose=False)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df_fe.drop(columns = ['Machine failure']),
                                                    df['Machine failure'],
                                                    test_size=0.33,
                                                    random_state=42,
                                                    stratify = df['Machine failure'])

train_pool = Pool(data=X_train, label=y_train)

In [5]:
CatBoost.fit(train_pool)

feature_importance = pd.DataFrame({'feature_importance': CatBoost.get_feature_importance(train_pool), 
              'feature_names': X_test.columns}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
              
check_result(CatBoost, X_test, y_test )

{'auc_score': 0.9094595805699947, 'f1_score': 0.931770654883567}

In [6]:
feature_importance.to_csv('../data/feature_imporstance_WS.csv',index = False)

### Find best amount of features

In [7]:
search_num_features(df_fe,feature_importance, step = 5)

  0%|          | 0/24 [00:00<?, ?it/s]

F1_score - 0.6734304006608839, num_features - 0, AUC_score = 0.6151303997132103


  4%|▍         | 1/24 [00:02<01:03,  2.75s/it]

F1_score - 0.8076319061320266, num_features - 1, AUC_score = 0.7482747804265998


  8%|▊         | 2/24 [00:05<01:01,  2.79s/it]

...

F1_score - 0.9290100951057022, num_features - 26, AUC_score = 0.9049952948557088


 96%|█████████▌| 23/24 [02:29<00:09,  9.68s/it]

F1_score - 0.9268971302323933, num_features - 26, AUC_score = 0.9048384567126725


100%|██████████| 24/24 [02:40<00:00, 10.05s/it]


Best F1_score - 0.9345060499209441, num_features - 26


## Test data with upsampling

### Find feature importance

In [8]:
CatBoost = CatBoostClassifier(verbose=False)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df_fe.drop(columns = ['Machine failure']),
                                                    df['Machine failure'],
                                                    test_size=0.33,
                                                    random_state=42,
                                                    stratify = df['Machine failure'])
X_train, y_train = upsampling(1,X_train, y_train)

train_pool = Pool(data=X_train, label=y_train)

In [15]:
CatBoost.fit(train_pool)

feature_importance = pd.DataFrame({'feature_importance': CatBoost.get_feature_importance(train_pool), 
              'feature_names': X_test.columns}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
              
check_result(CatBoost, X_test, y_test)

{'auc_score': 0.9180744757124933, 'f1_score': 0.9329922027290448}

In [16]:
feature_importance.to_csv('../data/feature_imporstance.csv',index = False)

### Find best amount of features

In [ ]:
search_num_features(df_fe, feature_importance,upsamp_func = True, step = 5)

  0%|          | 0/12 [00:00<?, ?it/s]

F1_score - 0.5134960492124424, num_features - 0, AUC_score = 0.7089756228714823


  8%|▊         | 1/12 [00:06<01:11,  6.48s/it]

F1_score - 0.9260620182828463, num_features - 1, AUC_score = 0.9260620182828464


 17%|█▋        | 2/12 [00:13<01:06,  6.61s/it]

...

F1_score - 0.9247648902821317, num_features - 61, AUC_score = 0.9174471231403479


 92%|█████████▏| 11/12 [01:57<00:13, 13.21s/it]

F1_score - 0.9220717326480798, num_features - 61, AUC_score = 0.912982837426062


100%|██████████| 12/12 [02:11<00:00, 13.44s/it]


Best F1_score - 0.9399027517255195, num_features - 61
